# Imports

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
import itertools
from sklearn.metrics import mean_squared_error
from math import sqrt
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Upload data

In [ ]:
data_test = pd.read_csv('/content/drive/MyDrive/New_stuff/8-s sem/VKR/OnlineCourseID.csv')
data_new = pd.read_csv('/content/drive/MyDrive/New_stuff/8-s sem/VKR/df_1954.csv')

In [ ]:
data_test = data_test.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])
data_new = data_new.drop(columns=['Unnamed: 0'])

In [ ]:
data_test.drop_duplicates(inplace=True)
data_new.drop_duplicates(inplace=True)

In [ ]:
data_new.shape

(1570, 22)

# Generating user ratings

In [ ]:
data_new.sample(5)

,id_from_roo,title,description,institution,platform,language,started_at,record_end_at,finished_at,rating,...,content,lectures_number,external_url,roc_url,has_certificate,credits,requirements,learning_outcome,competences,actual
1407,85ea98bce48c439eb3309f3a763d4862,Философия и методология науки,Курс «Философия и методология науки» посвящен ...,УрФУ,Открытое образование,Русский,05.09.2021,2021-12-26 00:00:00,2022-02-06,0.01,...,"Модуль 1. Введение \nТема 1. Что такое наука, ...",15,'https:\/\/openedu.ru\/course\/urfu\/PHILSCI\/...,https://online.edu.ru/public/course.xhtml?face...,Есть,0.0,Степень бакалавра,Выявлять философские проблемы науки в научном ...,способностью формировать ресурсно-информационн...,True
327,43d7c2a6-b283-47dd-9eb9-c04d6f759040,Игропрактика в образовании,Курс посвящен применению игровых технологий в ...,НИТУ «МИСиС»,Открытое образование,Русский,13.09.2021,2021-12-20 00:00:00,2022-01-28,0.00,...,Разделы курса:\n1. Введение (неделя 1)\n1.1. Ц...,10,"'https:\/\/openedu.ru\/course\/misis\/IGRO\/',...",https://online.edu.ru/public/course.xhtml?face...,Есть,2.0,NaN,"\nРазличать базовые понятияе и модели, применя...",NaN,True
1069,9b5ba1e9914f4449a5aba4b576abf75b,Расходометрия в нефтегазовой промышленности,"В России, образовательный курс по расходометри...",ТюмГУ,Эдуардо,Русский,11.09.2017,2017-10-10 00:00:00,2017-12-25,0.00,...,NaN,19,'https:\/\/lms.eduardo.studio\/courses\/course...,https://online.edu.ru/public/course.xhtml?face...,Есть,0.0,NaN,NaN,NaN,True
203,fc4e6dd1-9e6d-4e84-afa1-e9f2f5d1d8cf,Введение в философию русского космизма,Курс адресован по преимуществу сильным учащимс...,Финансовый университет,Открытая онлайн-академия Финансового университета,Русский,06.09.2021,2021-12-31 00:00:00,2022-01-15,0.00,...,\nТема 1. Учение Н.Ф. Фёдорова. Основные идеи ...,25,'https:\/\/online.fa.ru\/courses\/course\-v1:f...,https://online.edu.ru/public/course.xhtml?face...,Есть,3.0,Данный онлайн-курс адресован по преимуществу с...,NaN,"способность осуществлять поиск, критический ан...",True
434,380af6b745c24e31812f60c028b275c0,История науки и техники в области приборостроения,Курс об истории развития приборостроения и тех...,"СПбГЭТУ ""ЛЭТИ""",Лекториум,Русский,Дата старта уточняется,NaN,NaN,0.00,...,NaN,31,"'https:\/\/www.lektorium.tv\/mooc2\/31281','_s...",https://online.edu.ru/public/course.xhtml?face...,Есть,0.0,NaN,NaN,NaN,True


In [ ]:
data_new.isnull().sum()

id_from_roo            0
title                  0
description            4
institution            0
platform               0
language               0
started_at             0
record_end_at        774
finished_at          662
rating                 0
visitors_number        0
duration               0
content              272
lectures_number        0
external_url           0
roc_url                0
has_certificate        0
credits                0
requirements         487
learning_outcome    1235
competences         1368
actual                 0
dtype: int64

In [ ]:
data_new['rating'].unique()

array([0.  , 0.03, 0.05, 0.01, 0.02, 0.1 , 0.16, 0.07, 0.04, 0.29, 0.09,
       3.7 , 1.49, 0.22, 0.15, 0.13, 0.2 , 0.06, 2.55, 0.08, 1.13, 0.17,
       0.19, 0.18, 1.11, 0.25, 2.2 , 0.88, 1.99, 0.6 , 0.11, 2.51, 4.62,
       0.85, 2.67, 2.49, 2.43, 0.58, 0.14, 1.15, 1.12, 0.21])

In [ ]:
data_new['rating'].unique().sum()

37.019999999999996

In [ ]:
data_new.columns

Index(['id_from_roo', 'title', 'description', 'institution', 'platform',
       'language', 'started_at', 'record_end_at', 'finished_at', 'rating',
       'visitors_number', 'duration', 'content', 'lectures_number',
       'external_url', 'roc_url', 'has_certificate', 'credits', 'requirements',
       'learning_outcome', 'competences', 'actual'],
      dtype='object')

In [ ]:
values = []

for i in range(0,150000):
  rand = (np.random.randint(0, 1570))
  values.append([data_new['id_from_roo'][rand], data_new['title'][rand], data_new['description'][rand], np.random.randint(0, 1000), np.random.randint(1, 6)])

In [ ]:
new_df = pd.DataFrame(values, columns=['course_id', 'title', 'description', 'user_id', 'rating'])
new_df = new_df.sort_values('user_id')
key = ['course_id', 'user_id']
new_df = new_df.drop_duplicates(subset=key)
new_df

,course_id,title,description,user_id,rating
104014,8125c50d-b86b-44b1-9c6b-fb3dd2acc5c7,Образовательный форсайт,Модуль мобильности.В рамках курса предполагает...,0,1
131761,dcaef48eb5ee45789581629bb4fc1740,Basics of project management,"The course ""Fundamentals of Project Management...",0,1
131606,ebb8f0a5-a4be-4e8b-a726-75df14963c1d,Логистика технологических процессов и производств,Курс «Логистика технологических процессов и пр...,0,5
98765,39207c5b-d870-40da-9904-e88ff6f72196,Экономика (для студентов непрофильных направле...,Курс предназначен и адаптирован для студентов ...,0,2
89894,e7302de8-9b1d-41eb-ab13-dcaa16fdccd9,Методология и методы научных исследований в ...,Курс познакомит слушателей методологией и мето...,0,1
...,...,...,...,...,...
98580,20b1761d-2417-433b-8cae-b071514e98c3,Вспомнить все! Школьная математика для первоку...,В курсе рассматриваются основные темы математи...,999,4
96215,f45ead96dd1c43feab3913689a01b5cc,Логистика,"Цепь поставок — это обширная сеть, охватывающа...",999,4
80132,6c1ecef9-163a-4f26-8e0f-700d2efe3ebb,Цифровая культура,Целью курса является знакомство с общей концеп...,999,5
1514,5bde4ffc65db4dcd8568c0f6703e12e9,Философия и история науки и техники,Курс «Философия и история науки и техники» пос...,999,2


In [ ]:
new_df.to_csv('/content/drive/MyDrive/New_stuff/8-s sem/VKR/Gen_data.csv')


# Test Recc data shaping

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/New_stuff/8-s sem/VKR/Gen_data.csv')

In [ ]:
data = data.drop(columns=['Unnamed: 0'])
data.drop_duplicates(inplace=True)

In [ ]:
data

,course_id,title,description,user_id,rating
0,8125c50d-b86b-44b1-9c6b-fb3dd2acc5c7,Образовательный форсайт,Модуль мобильности.В рамках курса предполагает...,0,1
1,dcaef48eb5ee45789581629bb4fc1740,Basics of project management,"The course ""Fundamentals of Project Management...",0,1
2,ebb8f0a5-a4be-4e8b-a726-75df14963c1d,Логистика технологических процессов и производств,Курс «Логистика технологических процессов и пр...,0,5
3,39207c5b-d870-40da-9904-e88ff6f72196,Экономика (для студентов непрофильных направле...,Курс предназначен и адаптирован для студентов ...,0,2
4,e7302de8-9b1d-41eb-ab13-dcaa16fdccd9,Методология и методы научных исследований в ...,Курс познакомит слушателей методологией и мето...,0,1
...,...,...,...,...,...
143171,20b1761d-2417-433b-8cae-b071514e98c3,Вспомнить все! Школьная математика для первоку...,В курсе рассматриваются основные темы математи...,999,4
143172,f45ead96dd1c43feab3913689a01b5cc,Логистика,"Цепь поставок — это обширная сеть, охватывающа...",999,4
143173,6c1ecef9-163a-4f26-8e0f-700d2efe3ebb,Цифровая культура,Целью курса является знакомство с общей концеп...,999,5
143174,5bde4ffc65db4dcd8568c0f6703e12e9,Философия и история науки и техники,Курс «Философия и история науки и техники» пос...,999,2


In [ ]:
data.shape

(143176, 5)

In [ ]:
data.columns

Index(['course_id', 'title', 'description', 'user_id', 'rating'], dtype='object')

In [ ]:
data_random = data.sample(1000)
data_random.user_id.value_counts()

298    5
891    5
172    4
65     4
91     4
      ..
270    1
481    1
334    1
718    1
414    1
Name: user_id, Length: 644, dtype: int64

In [ ]:
data_random.course_id.value_counts()

a55cea5d61174a50a8f6355c77b69c9f        4
2948658b-5195-44e4-bf1c-b0b98ede5edc    4
7df6bec817484a8dbf5a9645aebebcd5        4
23e66670998c424588cdac806ce955ca        4
d05f1a2b-03a0-4263-93b9-7f5b2a080c92    4
                                       ..
95a5fd1bdd5140f4b55ff39ddefa7b96        1
1af53d4a-3d22-4b6c-a928-61aae0b852dc    1
dfff323a-c5ee-490d-9f7e-32dd152b4b67    1
c22d728b-587a-4c1c-902e-84d7222a0851    1
5f6df58c347d457898b35eb93423692d        1
Name: course_id, Length: 734, dtype: int64

In [ ]:
train_data = pd.DataFrame()
test_data = pd.DataFrame()

In [ ]:
for user in data_random.user_id.value_counts().index:
  user_data = data[(data.user_id==user)&(data.course_id.isin(data_random.course_id.unique()))]
  train = user_data[:round(len(user_data)*0.7)]
  test = user_data[(round(len(user_data)*0.7)+1):]
  train_data = pd.concat([train_data, train], axis=0)
  test_data = pd.concat([test_data, test], axis=0)

In [ ]:
train_data.shape

(30520, 5)

In [ ]:
test_data.shape

(12462, 5)

In [ ]:
train_data.to_csv('/content/drive/MyDrive/New_stuff/8-s sem/VKR/train_data.csv')
test_data.to_csv('/content/drive/MyDrive/New_stuff/8-s sem/VKR/test_data.csv')

# Getting Recc

In [ ]:
train_data = pd.read_csv('/content/drive/MyDrive/New_stuff/8-s sem/VKR/train_data.csv')
test_data = pd.read_csv('/content/drive/MyDrive/New_stuff/8-s sem/VKR/test_data.csv')

In [ ]:
prediction_for_test = pd.DataFrame()
for i in train_data.user_id.unique():
  print(i)
  prediction = pd.DataFrame()
  user_courses = train_data[train_data.user_id == i] # Курсы пользователя
  eq_courses = train_data[train_data.course_id.isin(user_courses.course_id)].user_id.unique() # Пользователи, которые проходили такие же курс
  per_dict = {}
  user_group = train_data[train_data.course_id.isin(user_courses.course_id)].groupby(['user_id'])
  for u in eq_courses:
    person2 = user_group.get_group(u).sort_values(by='course_id')
    person1 = user_courses[user_courses['course_id'].isin(user_group.get_group(u)['course_id'])].sort_values(by='course_id')
    if (len(person1.rating) > 1) & (len(person2.rating) > 1):
      if pearsonr(person1.rating, person2.rating)[0] > 0.0: # Оставляем только пользователей с неотрицательной корреляцией
        per_dict[u] = pearsonr(person1.rating, person2.rating)[0]
  per_dict = {k: v for k, v in sorted(per_dict.items(), key=lambda item: item[1], reverse=True)} # Словарь сходства
  per_dict = dict(itertools.islice(per_dict.items(), 7)) # Оставляем только топ-7 сходств
  pearsonDF = pd.DataFrame.from_dict(per_dict, orient='index') # Приводим к датасету, чтобы посмотреть оценки курсов
  pearsonDF.columns = ['similarityIndex']
  pearsonDF['user_id'] = pearsonDF.index
  pearsonDF.index = range(len(pearsonDF))

  # Объединяем датасеты
  topUsersRating = pearsonDF.merge(train_data, left_on='user_id', right_on='user_id', how='inner')
  
  # Фильтруем по курсам, которым нужно поставить оценки
  test_courses = test_data[test_data.user_id == i+1].course_id # Есть у пользователя в test = нужно поставить оценку
  topUsersRating = topUsersRating[topUsersRating.course_id.isin(test_courses)]

  # Считаем оценку
  topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']

  tempTopUsersRating = topUsersRating.groupby('course_id').sum()[['similarityIndex','weightedRating']]
  tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']

  # Составляем датасет, который будем оценивать с тестом
  # Нужно получить оценку только по тем курсам, которые есть в test
  prediction['score'] = round(tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex'],2)
  #prediction['course_id'] = tempTopUsersRating.index
  prediction['user_id'] = i+1
  prediction_for_test = pd.concat([prediction_for_test, prediction])

298


/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


891
172
65
91
671
348
936
395
669
491
540
402
684
737
838
742
557
93
389
343
985
575
205
954
291
843
367
434
229
78
773
752
783
834
633
698
927
944
900
508
551
952
380
610


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


196
57
600
420
734
85
528
183
785
971
714
984
953
398
224
14
874
89
570
241
959
892
494
209
486
780
383
568
909
458
189
817
591
814
112


KeyboardInterrupt: ignored

In [ ]:
prediction_for_test

,score,user_id
09ac1c10250d41d99181a5a97d3d43fb,3.0,299
232c6c5e1b394921a58e968c4c01ba55,1.0,299
681082a8b4c347f4b02269d2c990a19b,4.0,299
80c212f0-217e-4d56-91f6-0dd0049be230,5.0,299
8157298952274e428d93c0535b58c03b,2.0,299
...,...,...
3a6eff55770d4b9888b9577c87c960dd,2.0,815
a6d5f6fa-0564-42ea-97be-fa43bdb23ed7,1.0,815
b7ce81acb50947ac89d5a14dbf5c8df8,2.5,815
cb4cd1a1-151d-4d47-8515-7fd5277e3797,4.0,815


In [ ]:
pred_dict = prediction_for_test.groupby(['user_id']).groups

In [ ]:
y_pred = prediction_for_test.score.to_list()
for i in range(len(y_pred)):
  if y_pred[i] - round(y_pred[i]) > 0.26:
    y_pred[i] = round(y_pred[i]) + 0.5
  else:
    y_pred[i] = float(round(y_pred[i]))

In [ ]:
y_pred

[3.0,
 1.0,
 4.0,
 5.0,
 2.0,
 2.5,
 2.0,
 4.0,
 2.0,
 1.0,
 4.0,
 2.0,
 2.0,
 3.0,
 3.0,
 2.0,
 2.0,
 5.0,
 5.0,
 2.0,
 3.0,
 4.0,
 3.0,
 4.0,
 3.0,
 4.0,
 3.0,
 3.0,
 1.0,
 3.0,
 2.0,
 5.0,
 1.0,
 5.0,
 4.0,
 1.0,
 5.0,
 1.0,
 2.0,
 2.0,
 2.0,
 2.0,
 3.0,
 5.0,
 3.0,
 5.0,
 4.0,
 1.0,
 4.0,
 4.0,
 1.0,
 3.0,
 4.0,
 1.0,
 5.0,
 5.0,
 2.0,
 2.0,
 3.0,
 1.0,
 5.0,
 4.0,
 4.0,
 3.0,
 4.0,
 3.0,
 3.0,
 2.5,
 1.0,
 1.0,
 1.0,
 5.0,
 4.5,
 3.0,
 2.0,
 2.0,
 5.0,
 1.0,
 3.0,
 3.0,
 4.0,
 2.0,
 5.0,
 2.0,
 5.0,
 4.0,
 3.0,
 1.0,
 1.0,
 1.0,
 1.0,
 2.0,
 2.0,
 1.0,
 1.0,
 2.5,
 3.0,
 3.0,
 1.0,
 4.0,
 2.0,
 4.0,
 1.0,
 2.0,
 3.0,
 1.0,
 5.0,
 2.0,
 3.0,
 4.0,
 4.0,
 1.0,
 5.0,
 5.0,
 5.0,
 4.0,
 5.0,
 2.0,
 4.0,
 4.0,
 2.5,
 3.0,
 5.0,
 4.0,
 2.0,
 5.0,
 1.0,
 4.0,
 5.0,
 3.0,
 1.0,
 4.0,
 2.0,
 3.0,
 1.0,
 1.0,
 3.0,
 3.0,
 5.0,
 3.0,
 4.0,
 3.0,
 2.0,
 1.0,
 4.0,
 5.0,
 1.0,
 1.0,
 4.0,
 4.0,
 5.0,
 1.0,
 3.0,
 1.0,
 4.0,
 3.0,
 2.0,
 2.0,
 5.0,
 2.0,
 3.0,
 4.0,
 2.0,
 4.5,
 3.0,
 1.0,
 5.0

In [ ]:
test_data

,Unnamed: 0,course_id,title,description,user_id,rating
0,42562,fba0820d-b172-4c09-9ed3-fa8c5718cd0b,Паровые турбины АЭС. Часть 1. Теория теплового...,"Курс ""Паровые турбины АЭС. Часть 1. Теория теп...",298,1
1,42566,c018a49a6cef4fa99a84d50ddef0f327,Начертательная геометрия,"Курс ""Начертательная геометрия"" рассматривает ...",298,3
2,42567,755bc46f18bf45daacccfafea437d9e6,Суперкомпьютеры и параллельная обработка данных,Данный курс посвящен одной из основных тенденц...,298,2
3,42568,f9432a39-f0b8-45d5-aca3-cb6ac53f9852,Подготовка экспертного заключения,В курсе рассматриваются такие темы как: цели и...,298,2
4,42570,d8dfc132d33c4c3db2bc8bb67d6855dd,Ценные бумаги: правовое регулирование,ЦЕЛЬ КУРСА: сформировать целостную картину рег...,298,3
...,...,...,...,...,...,...
12457,59390,b46be2ef8403457db9d7a15a9b499704,Нормативно-правовое обеспечение онлайн-обучени...,Онлайн-курс посвящен изучению вопросов нормати...,414,2
12458,59394,96e6d9c8569c4dfca08fad366c461132,Дискретная математика,Дискретная математика - одна из важнейших сост...,414,4
12459,59397,1577f5bee3d84de7b6f11dcae66a32a6,Интеллектуальные игры,"Интеллектуальные игры — это игры ума, игры с у...",414,2
12460,59398,ebb8f0a5-a4be-4e8b-a726-75df14963c1d,Логистика технологических процессов и производств,Курс «Логистика технологических процессов и пр...,414,1


In [ ]:
# Отсортируем только те курсы, по которым есть оценка
pred_test_data = pd.DataFrame()
for key in pred_dict.keys():
  user_test_data = test_data[(test_data.user_id == key) & (test_data.course_id.isin(pred_dict.get(key).to_list()))]
  pred_test_data = pd.concat([pred_test_data, user_test_data])

In [ ]:
y_true = pred_test_data.sort_values(by=['user_id', 'course_id']).rating.to_list()

In [ ]:
y_true

[5,
 3,
 2,
 2,
 1,
 1,
 2,
 1,
 2,
 1,
 5,
 5,
 3,
 3,
 1,
 2,
 2,
 1,
 1,
 3,
 2,
 5,
 1,
 2,
 2,
 2,
 4,
 1,
 5,
 2,
 5,
 4,
 2,
 2,
 5,
 5,
 3,
 4,
 3,
 4,
 5,
 5,
 2,
 5,
 3,
 1,
 4,
 5,
 2,
 5,
 1,
 3,
 4,
 1,
 2,
 5,
 2,
 3,
 2,
 3,
 4,
 2,
 2,
 2,
 3,
 3,
 4,
 5,
 1,
 5,
 1,
 3,
 4,
 3,
 3,
 4,
 1,
 2,
 4,
 2,
 2,
 2,
 5,
 4,
 3,
 4,
 1,
 1,
 5,
 4,
 2,
 3,
 1,
 1,
 3,
 2,
 5,
 2,
 2,
 5,
 1,
 2,
 1,
 5,
 3,
 5,
 3,
 5,
 1,
 3,
 3,
 2,
 3,
 5,
 5,
 5,
 3,
 2,
 2,
 2,
 2,
 5,
 5,
 5,
 2,
 2,
 3,
 1,
 3,
 5,
 1,
 1,
 4,
 3,
 1,
 1,
 3,
 4,
 3,
 3,
 3,
 3,
 2,
 2,
 5,
 4,
 1,
 4,
 4,
 3,
 1,
 1,
 5,
 1,
 5,
 3,
 1,
 2,
 1,
 1,
 3,
 4,
 5,
 1,
 2,
 5,
 2,
 1,
 4,
 3,
 3,
 5,
 5,
 1,
 5,
 2,
 3,
 3,
 2,
 3,
 1,
 3,
 4,
 5,
 2,
 4,
 4,
 3,
 3,
 3,
 4,
 2,
 1,
 2,
 1,
 3,
 2,
 1,
 4,
 2,
 5,
 3,
 1,
 5,
 2,
 3,
 1,
 2,
 2,
 1,
 5,
 1,
 3,
 5,
 3,
 4,
 3,
 5,
 3,
 2,
 3,
 1,
 3,
 3,
 5,
 4,
 1,
 2,
 3,
 2,
 2,
 2,
 4,
 4,
 4,
 1,
 3,
 3,
 4,
 4,
 1,
 5,
 2,
 5,
 2,
 2,
 4,
 4,
 1,
 2,


In [ ]:
# MSE
mean_squared_error(y_true, y_pred)

3.7506393861892584

In [ ]:
# RMSE
sqrt(mean_squared_error(y_true, y_pred))

1.936656754871461

# Getting Recc Text

In [ ]:
data

NameError: ignored

In [ ]:
train_data = pd.read_csv('/content/drive/MyDrive/New_stuff/8-s sem/VKR/train_data.csv')

In [ ]:
data_new = pd.read_csv('/content/drive/MyDrive/New_stuff/8-s sem/VKR/df_1954.csv')
data_new = data_new.rename(columns={"id_from_roo": "course_id"})
data_new.columns

Index(['Unnamed: 0', 'course_id', 'title', 'description', 'institution',
       'platform', 'language', 'started_at', 'record_end_at', 'finished_at',
       'rating', 'visitors_number', 'duration', 'content', 'lectures_number',
       'external_url', 'roc_url', 'has_certificate', 'credits', 'requirements',
       'learning_outcome', 'competences', 'actual'],
      dtype='object')

In [ ]:
data_new = data_new.drop(columns=['Unnamed: 0', 'institution', 'platform', 'language', 'started_at', 'record_end_at',
                                  'finished_at', 'platform', 'language', 'started_at', 'record_end_at', 'finished_at',
                                  'rating', 'visitors_number', 'duration', 'content', 'lectures_number', 'external_url',
                                  'roc_url', 'has_certificate', 'credits', 'requirements', 'learning_outcome', 'competences', 'actual'])
data_new.drop_duplicates(inplace=True)

In [ ]:
data_new.columns

Index(['course_id', 'title', 'description'], dtype='object')

In [ ]:
data_new['title_for_rec'] = data_new.title.apply(lambda x: re.sub(r' ', ',', x) )
data_new.head()

,course_id,title,description,title_for_rec
0,43706abc-6f19-432e-8520-0bc88e8736ff,2D+3D проектирование в AutoCAD,Процесс создания чертежей в инженерной и архит...,"2D+3D,проектирование,в,AutoCAD"
1,b947160f-0142-49ef-acfd-bdc4cb3d01e7,Actual Issues of General Psychiatry,"The course ""Actual Issues of General Psychiatr...","Actual,Issues,of,General,Psychiatry"
2,b060cad4-0b8c-471f-9e95-697587649283,Advanced nanomaterials science (Nanomagnetism...,The main goal in the development of the di...,"Advanced,nanomaterials,science,,(Nanomagnetism..."
3,c6968df0-6a51-49e6-b64f-4811daa2eee5,Anatomy of the skeletal system,"The course ""Anatomy of the skeletal system"" co...","Anatomy,of,the,skeletal,system"
4,a96b4fa2-7ef8-47e4-b11c-b964933178fc,Application of magnetic materials,The course “Application of magnetic materia...,"Application,of,magnetic,materials"


In [ ]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(2, 5), min_df=1, stop_words='english')
tfidf_matrix = tf.fit_transform(data_new['title_for_rec'])
tfidf_matrix.shape

(1570, 8337)

In [ ]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
cosine_sim

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.26319513,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.26319513, 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [ ]:
titles = data_new['title']
indices = pd.Series(data_new.index, index=data_new['title'])

In [ ]:
def recommendations_for_user(user_data):
  user_data_high = user_data[user_data.rating >= 4]
  recommend_courses = pd.DataFrame()
  for title in user_data_high.title:
    try:
      idx = indices[title]
      sim_scores = list(enumerate(cosine_sim[idx]))
      sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
      sim_scores = sim_scores[0:10]
      course_indices = [i[0] for i in sim_scores]
      rec_courses = data_new[data_new.title.isin(titles.iloc[course_indices])]
      rec_courses = rec_courses[~rec_courses.title.isin(user_data.title)]
      recommend_courses = pd.concat([recommend_courses, rec_courses])
    except:
      continue
  recommend_courses = recommend_courses[~recommend_courses.course_id.isin(user_data.course_id)]
  recommend_courses = recommend_courses.drop_duplicates()
  return recommend_courses

In [ ]:
train_data_m = pd.merge(train_data, data_new[['course_id', 'title_for_rec']], on='course_id')
# train_data_m = pd.merge(train_data, data_new[['course_id', 'title']], on='course_id')

In [ ]:
train_data_m

,Unnamed: 0,course_id,title,description,user_id,rating,title_for_rec
0,42448,ef4dec0e-ac65-4e24-bfa6-d3fb2efc7cec,Основы антикризисной разработки корпоративных ...,Корпоративные информационные системы – один са...,298,1,"Основы,антикризисной,разработки,корпоративных,..."
1,29315,ef4dec0e-ac65-4e24-bfa6-d3fb2efc7cec,Основы антикризисной разработки корпоративных ...,Корпоративные информационные системы – один са...,205,1,"Основы,антикризисной,разработки,корпоративных,..."
2,119541,ef4dec0e-ac65-4e24-bfa6-d3fb2efc7cec,Основы антикризисной разработки корпоративных ...,Корпоративные информационные системы – один са...,834,1,"Основы,антикризисной,разработки,корпоративных,..."
3,132847,ef4dec0e-ac65-4e24-bfa6-d3fb2efc7cec,Основы антикризисной разработки корпоративных ...,Корпоративные информационные системы – один са...,927,2,"Основы,антикризисной,разработки,корпоративных,..."
4,112047,ef4dec0e-ac65-4e24-bfa6-d3fb2efc7cec,Основы антикризисной разработки корпоративных ...,Корпоративные информационные системы – один са...,780,2,"Основы,антикризисной,разработки,корпоративных,..."
...,...,...,...,...,...,...,...
30515,142051,a11355a5d2f4481788106943db3b1442,Инженерная механика,Данный курс ориентирован на реализацию образов...,992,4,"Инженерная,механика"
30516,65961,a11355a5d2f4481788106943db3b1442,Инженерная механика,Данный курс ориентирован на реализацию образов...,460,4,"Инженерная,механика"
30517,81384,a11355a5d2f4481788106943db3b1442,Инженерная механика,Данный курс ориентирован на реализацию образов...,567,5,"Инженерная,механика"
30518,103662,a11355a5d2f4481788106943db3b1442,Инженерная механика,Данный курс ориентирован на реализацию образов...,723,5,"Инженерная,механика"


In [ ]:
train_data_m.user_id.value_counts()

333    67
729    64
941    62
528    60
637    59
       ..
110    34
145    34
249    34
27     34
534    34
Name: user_id, Length: 644, dtype: int64

In [ ]:
data_new.keys()

Index(['course_id', 'title', 'description', 'title_for_rec'], dtype='object')

In [ ]:
user_selected = train_data_m[train_data_m.user_id == 941]
user_selected_rec = recommendations_for_user(user_selected)
user_selected_rec

,course_id,title,description,title_for_rec
0,43706abc-6f19-432e-8520-0bc88e8736ff,2D+3D проектирование в AutoCAD,Процесс создания чертежей в инженерной и архит...,"2D+3D,проектирование,в,AutoCAD"
1,b947160f-0142-49ef-acfd-bdc4cb3d01e7,Actual Issues of General Psychiatry,"The course ""Actual Issues of General Psychiatr...","Actual,Issues,of,General,Psychiatry"
2,b060cad4-0b8c-471f-9e95-697587649283,Advanced nanomaterials science (Nanomagnetism...,The main goal in the development of the di...,"Advanced,nanomaterials,science,,(Nanomagnetism..."
3,c6968df0-6a51-49e6-b64f-4811daa2eee5,Anatomy of the skeletal system,"The course ""Anatomy of the skeletal system"" co...","Anatomy,of,the,skeletal,system"
4,a96b4fa2-7ef8-47e4-b11c-b964933178fc,Application of magnetic materials,The course “Application of magnetic materia...,"Application,of,magnetic,materials"
5,f841d96e49f9473892ac556add4864a4,Applied IR spectroscopy,The goals of the course are to give the basic ...,"Applied,IR,spectroscopy"
6,cf6da702-d171-445c-be40-c5d675df55b6,Arctic Images: Cinema Borealis (на английском ...,Данный курс направлен на изучение методов реп...,"Arctic,Images:,Cinema,Borealis,(на,английском,..."
7,dcaef48eb5ee45789581629bb4fc1740,Basics of project management,"The course ""Fundamentals of Project Management...","Basics,of,project,management"
8,1ddab5ad-fdb5-46bf-9631-ef66a6518473,Business writing and e-mailing,Курс создан для повышения уровня иноязычной ко...,"Business,writing,and,e-mailing"
230,563244b550df4978bfe5615a675db01d,Выдающиеся журналисты XX – XXI вв.,Курс посвящен развитию журналистики в России о...,"Выдающиеся,журналисты,XX,–,XXI,вв."


In [ ]:
len(user_selected_rec)

27

In [ ]:
test_data = pd.read_csv('/content/drive/MyDrive/New_stuff/8-s sem/VKR/test_data.csv')

In [ ]:
user_selected_test = test_data[test_data.user_id == 941]
len(user_selected_test)

26

In [ ]:
user_selected_test = user_selected_test[user_selected_test.course_id.isin(user_selected_rec.course_id)]

In [ ]:
user_selected_test

,Unnamed: 0,course_id,title,description,user_id,rating
11537,134875,e21def87309844c886841696a7238208,Аналитическое сопровождение онлайн-обучения,Одним из преимуществ использования онлайн-курс...,941,4
11553,134909,9985b4717a7f4f7fa1fe444a690eb125,Стилистика русского языка,Изучение образных средств языка повышает культ...,941,4
11555,134912,34d0058b4ba647529d738425055a26aa,Проектирование индивидуальных траекторий обуче...,Индивидуализация образовательной программы мож...,941,2
11557,134915,d3ebb605cb9b4f219ea324b642a8ac2a,Тьюторское сопровождение онлайн-обучения,В данном курсе на примере нашего опыта работы ...,941,2


In [ ]:
print(user_selected_test.rating.mean())
print(user_selected_test.rating.median())

3.0
3.0


# Recc Text New

In [ ]:
train_data = pd.read_csv('/content/drive/MyDrive/New_stuff/8-s sem/VKR/train_data.csv')

In [ ]:
data_new = pd.read_csv('/content/drive/MyDrive/New_stuff/8-s sem/VKR/df_1954.csv')
data_new = data_new.rename(columns={"id_from_roo": "course_id"})
data_new.columns

Index(['Unnamed: 0', 'course_id', 'title', 'description', 'institution',
       'platform', 'language', 'started_at', 'record_end_at', 'finished_at',
       'rating', 'visitors_number', 'duration', 'content', 'lectures_number',
       'external_url', 'roc_url', 'has_certificate', 'credits', 'requirements',
       'learning_outcome', 'competences', 'actual'],
      dtype='object')

In [ ]:
data_new = data_new.drop(columns=['Unnamed: 0', 'institution', 'platform', 'language', 'started_at', 'record_end_at',
                                  'finished_at', 'platform', 'language', 'started_at', 'record_end_at', 'finished_at',
                                  'rating', 'visitors_number', 'duration', 'content', 'lectures_number', 'external_url',
                                  'roc_url', 'has_certificate', 'credits', 'requirements', 'learning_outcome', 'competences', 'actual'])
data_new.drop_duplicates(inplace=True)

In [ ]:
data_new.columns

Index(['course_id', 'title', 'description'], dtype='object')

In [ ]:
data_new['description'][0]

'Процесс создания чертежей в инженерной и архитектурно-строительной деятельности полностью реализуется в специальных графических пакетах программ. Программа AutoCAD — многофункциональный программный продукт, реализующий 2D- и 3D-технологии представления проектов и изобретений практически в любой сфере деятельности. Этот курс поможет за короткое время овладеть навыками работы в AutoCADКурс занял I место на Международном конкурсе открытых онлайн-курсов EdCrunch Award OOC-2018 в номинации «Лучший онлайн-курс по версии экспертов»'

In [ ]:
tfidf = TfidfVectorizer(stop_words='english')
data_new['description'] = data_new['description'].fillna('')

#Построение необходимой матрицы TF-IDF путем приминения метода fit_transform на столбце с описаниями
description_matrix = tfidf.fit_transform(data_new['description'])
description_matrix.shape

(1570, 23898)

In [ ]:
similarity_matrix = linear_kernel(description_matrix, description_matrix)
similarity_matrix

array([[1.        , 0.        , 0.        , ..., 0.03179227, 0.02105537,
        0.00957766],
       [0.        , 1.        , 0.05757903, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.05757903, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.03179227, 0.        , 0.        , ..., 1.        , 0.03448307,
        0.04276832],
       [0.02105537, 0.        , 0.        , ..., 0.03448307, 1.        ,
        0.04120908],
       [0.00957766, 0.        , 0.        , ..., 0.04276832, 0.04120908,
        1.        ]])

In [ ]:
#Преоборазование индексов в data_new
mapping = pd.Series(data_new.index, index = data_new['title'])
mapping

title
2D+3D проектирование в AutoCAD                                     0
Actual Issues of General Psychiatry                                1
Advanced nanomaterials science  (Nanomagnetism fundamentals)       2
Anatomy of the skeletal system                                     3
Application of magnetic materials                                  4
                                                                ... 
Язык современной рекламы и СМИ                                  1565
Язык эффективной коммуникации                                   1566
Язык эффективной коммуникации в профессиональном общении        1567
Язык эффективной коммуникации в цифровой среде                  1568
Японский язык. Начальный уровень                                1569
Length: 1570, dtype: int64

In [ ]:
def recommend_courses_based_on_description(course_input):
  course_index = mapping[course_input]
  #Получение значений сходств с другими курсами
  #similarity_score – лист индексов и матрицы схожести
  similarity_score = list(enumerate(similarity_matrix[course_index]))
  #Сортировка в обратном порядке оценки сходства импортированного курса с другими курсами
  similarity_score = sorted(similarity_score, key=lambda x: x[1], reverse=True)
  #Отбор 15 самых похожих курсов по описанию и игнорирование первого курса в списке (сам импортированный курс)
  similarity_score = similarity_score[1:15]
  #Вывод названий курсов
  course_indices = [i[0] for i in similarity_score]
  return (data_new['title'].iloc[course_indices])

In [ ]:
recommend_courses_based_on_description('2D+3D проектирование в AutoCAD')

480              Компьютерная инженерная графика. Часть 1
347     Инновационная экономика и технологическое пред...
1234                 Теории денег. От ракушки до биткоина
908                             Петербургские перекрестки
1124    Свободное программное обеспечение для компьюте...
1354    Управление финансовой эффективностью образоват...
481              Компьютерная инженерная графика. Часть 2
1522    Экспертиза качества онлайн-курсов, используемы...
394     Использование онлайн-курсов при проектировании...
738     Организация и планирование учебного процесса в...
395     Использование современных технологий и методик...
743     Организация мониторинга качества реализации об...
1174    Создание нормативно-правового обеспечения для ...
760         Основы архитектуры и строительных конструкций
Name: title, dtype: object

In [ ]:
recommend_courses_based_on_description('Anatomy of the skeletal system')

25                                     General Histology
51     Physical and chemical properties of proteins a...
32                     International Portfolio Managment
58          Regenerative Medicine and Tissue Engineering
17     Determination launch vehicle flight characteri...
24                     Fundamentals of General Chemistry
37     Introduction to Neuroeconomics: How the Brain ...
50                           Myths and Facts About Rocks
29     Industrial Organization: Strategy and Competit...
789    Основы кристаллохимии | Introduction to Crysta...
55                                      Public Economics
68     The cell cycle and cell death. Kinetics of Bio...
67       The Basics of Business and Public Communication
34             Introduction to Enumerative Combinatorics
Name: title, dtype: object